In [ ]:
import os
import cv2
import matplotlib.pyplot as plt
import numpy as np
import numpy as np
import scipy as sc
from scipy.sparse.linalg import svds
from scipy.sparse import csr_matrix
from scipy import misc
from skimage.util import img_as_ubyte
import fnmatch
import time
import cv2

In [ ]:
def GaussianFilter(image):
    kernelsize = 3
    gaussian1d = cv2.getGaussianKernel(kernelsize,0)
    gaussian2d = (gaussian1d.T*gaussian1d)
    image = cv2.filter2D(image, -1, gaussian2d)
    image = cv2.normalize(image,None,0,255,cv2.NORM_MINMAX)
    image = -image + np.max(image)
    return image

In [ ]:
def ExtractRawChannels(image):
    
    # Raw images are arranged in a Bayer Filter pattern:
    # B|G|B|G
    # G|R|G|R
    # B|G|B|G
    # G|R|G|R
    # By using the raw image instead of the color-processed image, we can get a higher spatial resolution
    
    # Zero out all green/blue pixels, smooth resultant image with a Gaussian kernel
    red = image.copy()
    red[0::2,1::2] = 0
    red[1::2,:] = 0
    red[:,:,1:]=0
    #red = red - (np.max(red)/2)
    #red[red < 0] = 0
    #red = red.astype(np.uint8)
    red[:,:,0] = GaussianFilter(red[:,:,0])
    red = cv2.cvtColor(red,cv2.COLOR_BGR2GRAY)
    
    # Zero out all red/blue pixels, smooth resultant image with a Gaussian kernel
    green = image.copy()
    green[0::2,0::2] = 0
    green[1::2,1::2] = 0
    green[:,:,(0,2)]=0
    #green = green - (np.max(green)/2)
    #green[green < 0] = 0
    #green = green.astype(np.uint8)
    green[:,:,1] = GaussianFilter(green[:,:,1])
    green = cv2.cvtColor(green,cv2.COLOR_BGR2GRAY)
    
    # Zero out all red/green pixels, smooth resultant image with a Gaussian kernel
    blue = image.copy()
    blue[0::2,:] = 0
    blue[1::2,0::2] = 0
    blue[:,:,:2]=0
    #blue = blue - (np.max(blue)/2)
    #blue[blue < 0] = 0
    #blue = blue.astype(np.uint8)
    blue[:,:,2] = GaussianFilter(blue[:,:,2])
    blue = cv2.cvtColor(blue,cv2.COLOR_BGR2GRAY)
    
    return red, green, blue

In [ ]:
path = 'First Tests With Flow Chamber/Jan 26 2021'

In [ ]:
files = [file for file in os.listdir(path) if file.endswith('tif')]

if not os.path.exists(os.path.join(path,'Processed')):
    os.makedirs(os.path.join(path,'Processed','ChannelExtracted'))
    os.makedirs(os.path.join(path,'Processed','BackgroundRemoved'))
    os.makedirs(os.path.join(path,'Processed','FlowAnalyzed'))
    os.makedirs(os.path.join(path,'Processed','Particles'))

for file in files:
    name = file[:-4]
    image = cv2.imread(os.path.join(path,file))
    red, green, blue = ExtractRawChannels(image)
    cv2.imwrite(os.path.join(path,'Processed','ChannelExtracted',name+'_red.tif'),red)
    cv2.imwrite(os.path.join(path,'Processed','ChannelExtracted',name+'_green.tif'),green)
    cv2.imwrite(os.path.join(path,'Processed','ChannelExtracted',name+'_blue.tif'),blue)

print(len(files))

In [ ]:
# Background removal with SVD

#Input Variables
NumOfSubtractionModes = 1 #more modes means more time computing the SVD as well as doing background subtraction for each image
NumOfImagesPerSVDCycle = 108 #len(files) #must be an exact divisor of the total number of images in each frame(i.e. the run was 1500 captures, use 100, 150, 300, 500, etc.)
Frames = ['*red*','*green*','*blue*'] #Frame name keyphrases that it searches for


os.system('cls' if os.name == 'nt' else 'clear')
NumOfFrames = len(Frames)
FileNames = sorted(os.listdir(os.path.join(path,'Processed','ChannelExtracted')))

NumOfCycles = (len(FileNames)/(NumOfImagesPerSVDCycle*NumOfFrames))
for CycleNumber in range(1,(np.floor(NumOfCycles)).astype(int) + 1): # Cycle through all the files in NumOfImagesPerSVDCycle increments, np.floor is for odd numbers of files that don't match multiples of NumImagesPerSVDCycle
    CycleNames = FileNames[(((NumOfImagesPerSVDCycle*NumOfFrames)*CycleNumber)-(NumOfFrames*NumOfImagesPerSVDCycle)):((NumOfImagesPerSVDCycle*NumOfFrames)*CycleNumber)] #all the names for one cycle, read as 1LA,1LB,1RA,1RB,1TA,1TB,2LA,etc.
    for frame in range(NumOfFrames): # Cycle through all the frames in a given cycle
        FrameNames = fnmatch.filter(CycleNames,Frames[frame]) #find all the names for a given frame in the cycle
        NumberofFiles = len(FrameNames)
        ImageSize = np.shape(cv2.imread(os.path.join(path,'Processed','ChannelExtracted',FrameNames[0]))) #test the first image for size
        ImageSize = ImageSize[0:2]
        ImageVectors = np.empty([NumberofFiles,ImageSize[0] * ImageSize[1]],dtype=np.uint8) #initialization before loop
        print("\n--------------------------Reading Images---------------------------\n")
        for ImageNumber in range(NumberofFiles):
            print("Reading:",FrameNames[ImageNumber])
            image = cv2.imread(os.path.join(path,'Processed','ChannelExtracted',FrameNames[ImageNumber])).astype(np.uint8)
            ImageVectors[ImageNumber,:] = (np.ravel(image[:,:,0],"F")) #ravel/vectorize each 2D image to 1D and insert into the 2D matrix "ImageVectors"
        print("\n-----------------Performing Sparse SVD Algorithm-------------------\n")
        u, s, v = svds(csr_matrix.asfptype(np.transpose(ImageVectors)),k=NumOfSubtractionModes,which='LM')
        um,sm,vm = np.asmatrix(u),np.asmatrix(s),np.asmatrix(v) #move to matrix data type for processing
        del u,s,v #memory management
        SingleImageModes = np.empty([NumOfSubtractionModes,ImageSize[0],ImageSize[1]]) #initialization before loop
        print("\n--------------------------Writing Images---------------------------\n")
        for image in range(NumOfImagesPerSVDCycle):
            for modes in range(NumOfSubtractionModes):
                SingleImageModes[modes,:,:] = np.reshape(um[:,modes] * sm[:,modes] * vm[modes,image],[ImageSize[0],ImageSize[1]],order="F") #reshape the modes back into a stack of 2D background images
            FilteredImage = np.reshape(np.transpose(ImageVectors[image,:]),[ImageSize[0],ImageSize[1]],order="F") - np.sum(SingleImageModes,axis=0) #subtract the sum of all background modes from the original image
            FilteredImage[FilteredImage < 0] = 0 #clean up/delete all negative values, as those will wrap around when saving it in unsigned byte format
            STDFilter = 10*np.std(FilteredImage[FilteredImage != 0]) #10 is chosen arbitrarily for the data, it seems to work well for PIV data
            FilteredImage[FilteredImage > STDFilter] = STDFilter #remove all values that are greater than the standard deviation based filter (this helps to eliminate glare points in the image)
            print("Writing:",FrameNames[image])
            FilteredImage = FilteredImage / np.max(FilteredImage)
            cv2.imwrite(os.path.join(path,'Processed','BackgroundRemoved',FrameNames[image]),img_as_ubyte(FilteredImage)) #save the image in byte format
    timeleft = (time.clock() * ((np.floor(NumOfCycles)*NumOfFrames) / ( ((CycleNumber-1)*NumOfFrames)+(frame+1) )))-time.clock()
    os.system('cls' if os.name == 'nt' else 'clear')
    print("\nTime to completion:",format(timeleft/60, '.4f'),"minutes")
print('\n\n\nFinished! :)')


In [ ]:
def DetectBlobs(image):
    # Set our filtering parameters 
    # Initialize parameter settiing using cv2.SimpleBlobDetector 
    params = cv2.SimpleBlobDetector_Params()

    # Set Color filtering parameter
    params.filterByColor = True
    params.blobColor = 255

    # Set Area filtering parameters 
    params.filterByArea = True
    params.minArea = 25
    params.maxArea = 10000

    # Set Circularity filtering parameters 
    params.filterByCircularity = True 
    params.minCircularity = 0.2
    params.maxCircularity = 1

    # Set Convexity filtering parameters 
    params.filterByConvexity = True
    params.minConvexity = 0.2
    params.maxConvexity = 1

    # Set Inertia filtering parameters 
    params.filterByInertia = True
    params.minInertiaRatio = 0.2
    params.maxInertiaRatio = 1
    
    # Set Threshold filtering parameters
    params.minThreshold = np.mean(image)*0.5
    params.maxThreshold = 255
    params.thresholdStep = params.maxThreshold / 20

    # Create a detector with the parameters 
    detector = cv2.SimpleBlobDetector_create(params) 
    # Detect blobs 
    keypoints = detector.detect(cv2.cvtColor(image,cv2.COLOR_BGR2GRAY))
    points = [keypoints[idx].pt for idx in range(0, len(keypoints))]
    blank = np.zeros((1,1))
    blobsimage = cv2.drawKeypoints(image*0, keypoints, blank, (255,0,0), cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
    return blobsimage, points

In [ ]:
for file in os.listdir(os.path.join(path,'Processed','BackgroundRemoved')):
    name = file[:-4]
    image = cv2.imread(os.path.join(path,'Processed','BackgroundRemoved',file))
    blobimage, points = DetectBlobs(image)
    number_of_blobs = len(points) 
    text = "Number of Blobs: " + str(len(points)) 
    cv2.putText(blobimage, text, (20, 550), 
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 100, 255), 2) 
    plt.imsave(os.path.join(path,'Processed','Particles',name+'.tif'),blobimage)

In [ ]:
Flow_List = []
Acceleration_List = []

# params for ShiTomasi corner detection
feature_params = dict( maxCorners = 1000,
                       qualityLevel = 0.3,
                       minDistance = 7,
                       blockSize = 7 )

# Parameters for lucas kanade optical flow
lk_params = dict( winSize  = (40,40),
                  maxLevel = 2,
                  criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))

# Create some random colors
color = np.random.randint(0,255,(1000,3))

background_removed_path = os.path.join(path,'Processed','BackgroundRemoved')
files = [file for file in os.listdir(background_removed_path) if 'red' in file]
for file in files[0:100]:
    print(file)
    red_path = background_removed_path+'/'+file[:-7]+'red.tif'
    green_path = background_removed_path+'/'+file[:-7]+'green.tif'
    blue_path = background_removed_path+'/'+file[:-7]+'blue.tif'
    red = cv2.cvtColor(cv2.imread(red_path),cv2.COLOR_BGR2GRAY)
    green = cv2.cvtColor(cv2.imread(green_path),cv2.COLOR_BGR2GRAY)
    blue = cv2.cvtColor(cv2.imread(blue_path),cv2.COLOR_BGR2GRAY)
    channel_images = [red,green,blue]
    
    #p0 = cv2.goodFeaturesToTrack(channel_images[0], mask = None, **feature_params)

    # Create a mask image for drawing purposes
    #mask = np.zeros_like(channel_images[0])


    #hsv = np.zeros_like(cv2.imread(red_path))
    #hsv[...,1] = 255
    
    
    for channel in range(2):
        
        # calculate optical flow

        flow = cv2.calcOpticalFlowFarneback(channel_images[channel], channel_images[channel+1], None, 0.5, 3, 15, 3, 5, 1.2, 0)
        #mag, ang = cv2.cartToPolar(flow[...,0], flow[...,1])
        #hsv[...,0] = ang #*180/np.pi/2
        #hsv[...,2] = cv2.normalize(mag,None,0,255,cv2.NORM_MINMAX)
        #rgb = cv2.cvtColor(hsv,cv2.COLOR_HSV2BGR)
        
        Flow_List.append(flow)
        
        '''
        p1, st, err = cv2.calcOpticalFlowPyrLK(channel_images[channel], channel_images[channel+1], p0, None, **lk_params)

        # Select good points
        good_new = p1[st==1]
        good_old = p0[st==1]

        # draw the tracks
        for i,(new,old) in enumerate(zip(good_new,good_old)):
            a,b = new.ravel()
            c,d = old.ravel()
            mask = cv2.line(mask, (a,b),(c,d), color[i].tolist(), 2)
            frame = cv2.circle(channel_images[channel],(a,b),5,color[i].tolist(),-1)
        img = cv2.add(channel_images[channel],mask)

        # Now update the previous frame and previous points
        p0 = good_new.reshape(-1,1,2)
        '''

    #plt.imsave(os.path.join(path,'Processed','FlowAnalyzed',file[:-7]+'FlowField.tif'),rgb)
    #plt.imsave(os.path.join(path,'Processed','Particles',file[:-7]+'ParticleTracks.tif'),mask)

In [ ]:
Flow_Average = np.max(Flow_List,axis=0)
mag, ang = cv2.cartToPolar(Flow_Average[...,0], Flow_Average[...,1])
hsv = np.zeros_like(cv2.imread(red_path))
hsv[...,1] = 255
hsv[...,0] = ang #*180/np.pi/2
hsv[...,2] = cv2.normalize(mag,None,0,255,cv2.NORM_MINMAX)
rgb = cv2.cvtColor(hsv,cv2.COLOR_HSV2BGR)
plt.imsave(os.path.join(path,'Processed','FlowAnalyzed','FlowAverage.tif'),rgb)

In [ ]:
from sklearn.decomposition import FastICA

path = 'LED Flash Timing/Jan 23 2021 Dual RGB Flash'
files = [file for file in os.listdir(path) if file.endswith('tif')]
for file in files:
    name = file[:-4]
    if '12809' in name:
        print(os.path.join(path,file))
        image = cv2.imread(os.path.join(path,file))
        red, green, blue = ExtractRawChannels(image)
        image = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)

image_vec = image.ravel()
red_vec = red.ravel()
green_vec = green.ravel()
blue_vec = blue.ravel()

RGB_vectorized = np.array([image_vec,red_vec,green_vec,blue_vec]).T
print(RGB_vectorized.shape)

In [ ]:
from sklearn.datasets import load_digits
from sklearn.decomposition import FastICA
X, _ = load_digits(return_X_y=True)
transformer = FastICA(n_components=4,random_state=0)
RGB_separated = transformer.fit_transform(RGB_vectorized)
RGB_separated.shape

In [ ]:
image_vec = RGB_separated[:,0]
red_vec = RGB_separated[:,1]
green_vec = RGB_separated[:,2]
blue_vec = RGB_separated[:,3]

In [ ]:
image_separated = np.reshape(image_vec,image.shape)
red_separated = np.reshape(red_vec,red.shape)
green_separated = np.reshape(green_vec,green.shape)
blue_separated = np.reshape(blue_vec,blue.shape)

In [ ]:
cv2.imwrite(path+'/Separated0.tif',cv2.normalize(image_separated,None,0,255,cv2.NORM_MINMAX).astype(np.uint8))
cv2.imwrite(path+'/Separated1.tif',cv2.normalize(red_separated,None,0,255,cv2.NORM_MINMAX).astype(np.uint8))
cv2.imwrite(path+'/Separated2.tif',cv2.normalize(green_separated,None,0,255,cv2.NORM_MINMAX).astype(np.uint8))
cv2.imwrite(path+'/Separated3.tif',cv2.normalize(blue_separated,None,0,255,cv2.NORM_MINMAX).astype(np.uint8))